In [31]:
from collections import defaultdict
from typing import Optional, List


#### Bottom View Binary Tree (queue)
* similar to bfs

In [21]:
# Python3 program to print Bottom
# Source: https://www.geeksforgeeks.org/bottom-view-binary-tree/

# Tree node class
class Node:
    def __init__(self, key):
        self.val = key
        self.hd = 0 # horizontal distance from center root node
        self.left = None
        self.right = None

def binary_tree_bottom_view(root):
    """
    Use horizontal distance to determine order of bottom view
    """

    if root is None:
        return

    # at the end we will have the following left to right bottom view
    """
    {
        # horizontal dist: node.val
        -2: 5,
        -1: 10,
         0: 4,
         1: 14,
         2: 25,
    }
    """
    lookup = dict()

    # Queue to store tree nodes in level
    # order traversal
    queue = []

    # Assign initialized horizontal distance
    # value to root node and add it to the queue.
    root.hd = 0

    # In STL, append() is used enqueue an item
    queue.append(root)

    while queue:
        node = queue.pop(0)

        # We always want to update the lookup with
        # this node's position and value
        lookup[node.hd] = node.val

        # Add left child to queue with hd = hd - 1
        if node.left is not None:
            node.left.hd = node.hd - 1
            queue.append(node.left)

        # Add right child to queue with hd = hd + 1
        if node.right is not None:
            node.right.hd = node.hd + 1
            queue.append(node.right)

    # Sort the map based on increasing hd for left to right bottom view
    for i in sorted(lookup.keys()):
        print(lookup[i], end = ' ')

                      20
                    /    \
                  8       22
                /   \    /   \
              5      3  4    25
                    / \
                  10    14

In [22]:
# Driver Code
root = Node(20)
root.left = Node(8)
root.right = Node(22)
root.left.left = Node(5)
root.left.right = Node(3)
root.right.left = Node(4)
root.right.right = Node(25)
root.left.right.left = Node(10)
root.left.right.right = Node(14)
print("Bottom view of the given binary tree :")
binary_tree_bottom_view(root)

Bottom view of the given binary tree :
5 10 4 14 25 

#### Bottom View Binary Tree (hashmap)

In [29]:
class Node:
    def __init__(self, key = None,
                      left = None,
                     right = None):
        self.val = key
        self.left = left
        self.right = right

def bottom_view_hashmap(root):

    # key = relative horizontal distance of the node from root node and
    # value = pair containing node's value and its level
    """
    {
        # horizontal dist: (node.val, node's level)
        -2: (5, 2),
        -1: (10, 3,
         0: (4, 2)
         1: (14, 3),
         2: (25, 2),
    }
    """
    lookup = dict()

    bottom_view_hashmap_util(root, lookup, 0, 0)

    # print the bottom view
    for i in sorted(lookup.keys()):
        print(lookup[i][0], end = " ")

def bottom_view_hashmap_util(root, lookup, hd, level):

    if root is None:
        return

    # If current level is more than or equal
    # to maximum level seen so far for the
    # same horizontal distance or horizontal
    # distance is seen for the first time,
    # update the dictionary
    if hd in lookup:
        if level >= lookup[hd][1]:
            lookup[hd] = [root.val, level]
    else:
        lookup[hd] = [root.val, level]

    # recurse for left subtree by decreasing
    # horizontal distance and increasing
    # level by 1
    bottom_view_hashmap_util(root.left,
                             lookup,
                             hd - 1,
                             level + 1)

    # recurse for right subtree by increasing
    # horizontal distance and increasing
    # level by 1
    bottom_view_hashmap_util(root.right,
                             lookup,
                             hd + 1,
                             level + 1)

print("Bottom view of the given binary tree:")
bottom_view_hashmap(root)

Bottom view of the given binary tree:
5 10 4 14 25 

#### Top View Binary Tree
Source: http://code2begin.blogspot.com/2018/07/top-view-of-binary-tree.html

#### Find Leaves of Binary Tree

```text
Input: root = [1,2,3,4,5]
Output: [[4,5,3],[2],[1]]
Explanation:
[[3,5,4],[2],[1]] and [[3,4,5],[2],[1]] are also considered correct answers since per each level it does not matter the order on which elements are returned.

Example 2:

Input: root = [1]
Output: [[1]]
```

In [33]:
# https://leetcode.com/problems/find-leaves-of-binary-tree/

# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
class Solution:

    def bottom_view_hashmap(self, root):
        deleted_list = []
        def util(root, lookup, hd, level, parent = None, is_left = True):
            if root is None:
                return

            if hd in lookup:
                # we've already seen this hd before, if the level is greater or equal,
                # update the value at this horizontal distance to this node's value and its level
                if level >= lookup[hd][1]:
                    lookup[hd] = [root.val, level]
            else:
                # ensure the root's val and level are in the lookup
                lookup[hd] = [root.val, level]

            # this node has children, only its children should be in the lookup
            if root.left or root.right:
                del lookup[hd]

            # recurse toward the left and right nodes. Will return None if they don't exist
            # pass the parent node in and indicate if this node points left or right
            # so we can chop off this node if it's a leaf
            util(root.left, lookup, hd-1, level + 1, parent=root, is_left=True)
            util(root.right, lookup, hd+1, level + 1, parent=root, is_left=False)


            if not root.left and not root.right:
                # add this node to deleted list then delete it
                deleted_list.append((root.val, parent, is_left))

        lookup = {}
        util(root, lookup, 0, 0)
        # the bottom view
        print(f"Bottom view: {[value[0] for _, value in lookup.items()]}")
        return deleted_list


    def findLeaves(self, root: Optional[TreeNode]) -> List[List[int]]:

        result = []
        i = 0
        while root:
            temp_vals = []
            # get the current bottom view
            deletion_list = self.bottom_view_hashmap(root)
            # add vals to result
            for val, _, _ in deletion_list:
                temp_vals.append(val)
            result.append(temp_vals)
            # delete the nodes in the deletion list
            for val, parent, is_left in deletion_list:
                if parent and is_left:
                    parent.left = None
                elif parent and not is_left:
                    parent.right = None
                if not parent:
                    # we're at the root
                    root = None
        return result

Solution().findLeaves(root)

Bottom view: [5, 10, 4, 25]
Bottom view: [3, 22]
Bottom view: [8]
Bottom view: [20]


[[5, 10, 14, 4, 25], [3, 22], [8], [20]]

#### Find minimum depth

The minimum depth is the number of nodes along the shortest path from the root node down to the nearest leaf node.

```
       3
     /   \
    9     20
         /   \
       15     7

Input: root = [3,9,20,null,null,15,7]
Output: 2
3 and 9 are the nodes in the shortest path

Input: root = [2,null,3,null,4,null,5,null,6]
Output: 5
```

This is a direct application of bfs

In [77]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def __repr__(self):
        return str({
            "val": self.val if self.val else None,
            "left": self.left.val if self.left else None,
            "right": self.right.val if self.right else None,
        })

class Solution:
    def minDepth(self, root: Optional[TreeNode]) -> int:

        if root is None:
            return 0

        def bfs(visited: List[TreeNode], node: TreeNode):
            queue = []
            depth = 1
            queue.append((node, depth))

            while queue:
                # if neighbor is not visited, add it to the queue
                elem, depth = queue.pop(0)
                visited.append(elem)

                if elem.left is None and elem.right is None:
                    return depth

                if elem.left and elem.left not in visited:
                    queue.append((elem.left, depth + 1))
                if elem.right and elem.right not in visited:
                    queue.append((elem.right, depth + 1))
        visited = []
        min_depth = bfs(visited, root)
        return min_depth

root = TreeNode(3)
b = TreeNode(9)
c = TreeNode(20)
d = TreeNode(15)
e = TreeNode(7)

root.left = b
root.right = c
c.left = d
c.right = e
Solution().minDepth(root)

2

In [78]:
root = TreeNode(2)
b = TreeNode(3)
c = TreeNode(4)
d = TreeNode(5)
e = TreeNode(6)

root.right = b
b.right = c
c.right = d
d.right = e
Solution().minDepth(root)

5